In [ ]:
%sendtofile main.py
hub_ip = '192.168.2.2'
ssid   = 'xxx'
passwd = 'xxx'
def test_oled():
    oled.fill(0)
    oled.text('ABCDEFGH',  0,  0, 1) 
    oled.text('VO    ID',  0,  10, 1) 
    oled.text('12345678',  0,  20, 1) 
    oled.text('Disappear',  0,  30, 1) 
    oled.text('abcdefgh',  0,  40, 1)  
    oled.show()   
    
def do_connect():
    import network
    sta_if = network.WLAN(network.STA_IF)
    if sta_if.isconnected():
        print('Already connected')
        pass
    else:
        sta_if.active(True)
        print('connecting to network...')
        sta_if.connect(ssid, passwd)
        while not sta_if.isconnected():
            pass
    print('network config:', sta_if.ifconfig())

def welcome():
    oled.fill(0) 
    oled.text('Welcome~',  0,  20, 1)  
    oled.show()    
        
import ssd1306, socket
from machine import Pin, I2C, enable_irq, disable_irq
from time import sleep
   
OLED_SDA_PIN = 4
OLED_SCL_PIN = 5 
i2c = I2C(scl=Pin(OLED_SCL_PIN), sda=Pin( OLED_SDA_PIN)) 
# https://docs.micropython.org/en/latest/esp8266/tutorial/ssd1306.html
oled = ssd1306.SSD1306_I2C(64, 48, i2c)   

test_oled()
do_connect()
welcome() 

Left_Blue_Pin     = 15  # D8
#Right_Red_Pin    =       RST     
Right_Green_Pin   =  2  # D4    
Right_Yellow_Pin  =  0  # D3     


Pin(Left_Blue_Pin, Pin.IN).irq(trigger=Pin.IRQ_RISING, handler=\
                               lambda pin: remote.set_pin(Left_Blue_Pin) )    
Pin(Right_Yellow_Pin, Pin.IN).irq(trigger=Pin.IRQ_FALLING, handler=\
                                  lambda pin: remote.set_pin(Right_Yellow_Pin) )  
Pin(Right_Green_Pin, Pin.IN).irq(trigger=Pin.IRQ_FALLING, handler=\
                                  lambda pin: remote.set_pin(Right_Green_Pin) )  

class REMOTE():
    count = 0
    pin_pressed = None
    def set_pin(self, i):
        self.pin_pressed = i
        
remote = REMOTE()

def http_get(url):
    _, _, host, path = url.split('/', 3)
    s = socket.socket()
    s.connect((host, 80))
    req = \
    '''GET /{} HTTP/1.0\r\nHost: foo\r\n\r\nbar'''.format(path).encode()
    s.send(req) 
    s.close() 

while 1:
    if not isinstance(remote.pin_pressed, int): 
        sleep(1)
        continue 
    remote.count += 1
    if remote.pin_pressed == Left_Blue_Pin:
        http_get('http://'+hub_ip+'/lb110') 
        oled.fill(0)  
        #         '12345678' 
        oled.text(str(remote.count),  0, 0, 1)  
        oled.text('Bedroom',  0,  10, 1)    
        oled.text(' Light ',  0,  20, 1)        
        oled.show() 
    elif remote.pin_pressed == Right_Yellow_Pin:
        http_get('http://'+hub_ip+'/ac')  
        oled.fill(0) 
        oled.text(str(remote.count),  0, 0, 1)  
        #         '12345678'  
        oled.text('   AC   ',  0,  20, 1)      
        oled.show() 
    elif remote.pin_pressed == Right_Green_Pin:
        http_get('http://'+hub_ip+'/kvm')  
        oled.fill(0) 
        oled.text(str(remote.count),  0, 0, 1)  
        #         '12345678'  
        oled.text('  KVM   ',  0,  20, 1)      
        oled.show() 
    else:
        oled.fill(0) 
        oled.text(str(remote.count),  0, 0, 1)  
        #         '12345678'  
        oled.text(' Error ',  0,  20, 1)      
        oled.text(str(remote.pin_pressed),  0, 40, 1)  
        oled.show()  
         
    sleep(3) 
    oled.fill(0)  
    oled.text(str(remote.count),  0, 0, 1) 
    #         '12345678'  
    oled.text('Welcome',  0,  20, 1)       
    oled.show()      
    remote.pin_pressed = None  

In [ ]:
import os
try:
    os.remove('main.py')
except OSError:
    print('removed already')

In [ ]:
%disconnect

In [ ]:
%serialconnect --port=/dev/ttyUSB0 --baud=115200

In [ ]:
%fetchfile --print boot.py

In [ ]:
%lsmagic